In [ ]:
# Step 1: 必要なライブラリをインポート
import re
import urllib.request

# Step 2: 青空文庫の「こころ」テキストデータ（ルビあり）をダウンロード
url = "https://www.aozora.gr.jp/cards/000148/files/773_14560.html"
file_name = "kokoro_raw.html"
urllib.request.urlretrieve(url, file_name)

# Step 3: HTMLから本文テキストだけを取り出す
from bs4 import BeautifulSoup

with open(file_name, encoding='shift_jis') as f:
  soup = BeautifulSoup(f, "html.parser")
  # テキスト抽出 (<div class="main_text>") から
  text = soup.find("div", class_="main_text").get_text()

# Step 4: ルビ削除関数
def remove_ruby(text):
  text = re.sub(r'｜([^《]+)《[^》]+》', r'\1', text)
  text = re.sub(r'《[^》]+》', '', text)
  return text

# Step 5: ヘッダー・フッター・空行を削除
def clean_text(text):
  text = remove_ruby(text)
  # 不要な空行や記号を消す
  text = re.sub(r'\n\s*\n', '\n', text)
  text = re.sub(r'［＃.*?］', '', text)  # 青空注記
  return text.strip()

# Step 6: 実行して整形
cleaned = clean_text(text)

# Step 7: 保存(GPT学習用にテキストファイル化)
with open("kokoro_clean.txt", "w", encoding="utf-8") as f:
  f.write(cleaned)

print("整形完了！ファイル名： kokoro_clean.txt")

整形完了！ファイル名： kokoro_clean.txt


In [ ]:
# 夏目漱石「坊ちゃん」をダウンロード
url = "https://www.aozora.gr.jp/cards/000148/files/752_14964.html"
file_name = "botchan_raw.html"
urllib.request.urlretrieve(url, file_name)

# HTMLから本文を取り出す
with open(file_name, encoding='shift_jis') as f:
    soup = BeautifulSoup(f, "html.parser")
    text = soup.find("div", class_="main_text").get_text()

# テキスト整形関数を再利用
cleaned_botchan = clean_text(text)

# 保存
with open("botchan_clean.txt", "w", encoding="utf-8") as f:
    f.write(cleaned_botchan)

print("坊ちゃん整形完了！")


坊ちゃん整形完了！


In [ ]:
# 夏目漱石「正岡子規」をダウンロード
url = "https://www.aozora.gr.jp/cards/000148/files/1751_6496.html"
file_name = "masaoka_raw.html"
urllib.request.urlretrieve(url, file_name)

# HTMLから本文を取り出す
with open(file_name, encoding='shift_jis') as f:
    soup = BeautifulSoup(f, "html.parser")
    text = soup.find("div", class_="main_text").get_text()

# テキスト整形関数を再利用
cleaned_masaoka = clean_text(text)

# 保存
with open("masaoka_clean.txt", "w", encoding="utf-8") as f:
    f.write(cleaned_botchan)

print("正岡子規整形完了！")


正岡子規整形完了！


In [ ]:
# こころと、坊ちゃん、正岡子規のデータを結合
with open("kokoro_clean.txt", "r", encoding="utf-8") as f:
  kokoro_text = f.read()

with open("botchan_clean.txt", "r", encoding="utf-8") as f:
  botchan_text = f.read()

with open("masaoka_clean.txt", "r", encoding="utf-8") as f:
  masaoka_text = f.read()

# 結合
combined_text = kokoro_text + "\n" + botchan_text + "\n" + masaoka_text

# 保存
with open("combined_clean.txt", "w", encoding="utf-8") as f:
  f.write(combined_text)

print("データ結合完了!")

データ結合完了!


In [ ]:
# ファイル読み込み
with open("combined_clean.txt", encoding="utf-8") as f:
  text = f.read()

# ユニークな文字を取り出す
chars = sorted(list(set(text)))

# 空白文字を追加
chars.append(' ')
vocab_size = len(chars)

# 文字 ⇔ ID の辞書を作る
stoi = {ch: i for i, ch in enumerate(chars)}  # string to int
itos = {i: ch for ch, i in stoi.items()}       # int to string

# エンコード・デコード関数
def encode(s):
  return [stoi[c] for c in s]

def decode(ids):
  return ''.join([itos[i] for i in ids])

print(f"語彙サイズ（ユニークな文字数）: {vocab_size}")
print(f"例:「私」→ {encode('私')}, {decode(encode('私'))}")

語彙サイズ（ユニークな文字数）: 2465
例:「私」→ [1623], 私


In [ ]:
# バッチ作成用のコード
import torch

# データ全体の数値化（前処理済み）
data = torch.tensor(encode(text), dtype=torch.long)

# 訓練用と検証用に分ける（9:1くらい）
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

# シーケンス長（どれくらいの長さで学習するか）
block_size = 8

# バッチ作成関数
def get_batch(split, batch_size=4):
  data_split = train_data if split == 'train' else val_data
  ix = torch.randint(len(data_split) - block_size, (batch_size,))

  x = torch.stack([data_split[i:i+block_size] for i in ix])
  y = torch.stack([data_split[i+1:i+block_size+1] for i in ix])

  return x, y

In [ ]:
# バッチ作成
data = torch.tensor(encode(text), dtype=torch.long)

# 訓練用と検証用に分ける（9:1くらい）
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

# バッチ作成関数
def get_batch(split, batch_size=4):
    data_split = train_data if split == 'train' else val_data
    ix = torch.randint(len(data_split) - block_size, (batch_size,))

    x = torch.stack([data_split[i:i+block_size] for i in ix])
    y = torch.stack([data_split[i+1:i+block_size+1] for i in ix])

    return x, y

In [ ]:
# テスト
x, y = get_batch('train')

for i in range(4):
  print("入力: ", decode(x[i].tolist()))
  print("正解: ", decode(y[i].tolist()))
  print()

入力:  るつもりかと云う
正解:  つもりかと云う権

入力:  要のない位地に立
正解:  のない位地に立っ

入力:  イカラ頭、乗るは
正解:  カラ頭、乗るは自

入力:  てくると、宿の亭
正解:  くると、宿の亭主



In [ ]:
# 1. モデルの作成
import torch
import torch.nn as nn

# モデルクラス
class CharRNN(nn.Module):
  def __init__(self, vocab_size, embed_size, hidden_size, block_size):
    super(CharRNN, self).__init__()
    self.block_size = block_size
    self.embedding = nn.Embedding(vocab_size, embed_size)
    self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
    self.fc = nn.Linear(hidden_size, vocab_size)

  def forward(self, x, h0=None):
    # 埋め込み層
    x = self.embedding(x)
    # RNN層
    out, h0 = self.rnn(x, h0)
    # 最終層の出力
    out = self.fc(out)
    return out, h0



In [ ]:
# 2. モデルの初期化
# ハイパーパラメータ
vocab_size = len(chars)  # 使用する文字の数
embed_size = 64  # 埋め込み層のサイズ
hidden_size = 512  # 隠れ層のサイズ
block_size = 8  # 入力サイズ

# モデルの初期化
model = CharRNN(vocab_size, embed_size, hidden_size, block_size)

# 損失関数 (CrossEntropyLoss)
loss_fn = nn.CrossEntropyLoss()

# 最適化関数 (Adam)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# 3. 学習の実行
epochs = 10000
for epoch in range(epochs):
  model.train()
  optimizer.zero_grad()  # 勾配の初期化

  # バッチデータを取得
  x_batch, y_batch = get_batch('train')  # 'train'を使って訓練データを取得

  # モデルの予測
  output, _ = model(x_batch)

  # 損失計算
  loss = loss_fn(output.view(-1, vocab_size), y_batch.view(-1))

  # 誤差逆伝播
  loss.backward()

  # パラメータの更新
  optimizer.step()

  if epoch % 100 == 0:  # 100回ごとに損失を表示
    print(f"Epoch {epoch}, Loss: {loss.item()}")

Epoch 0, Loss: 7.794919490814209
Epoch 100, Loss: 5.317493915557861
Epoch 200, Loss: 4.871608734130859
Epoch 300, Loss: 5.479851722717285
Epoch 400, Loss: 4.299712657928467
Epoch 500, Loss: 4.312597751617432
Epoch 600, Loss: 4.473081111907959
Epoch 700, Loss: 4.257058620452881
Epoch 800, Loss: 3.442563772201538
Epoch 900, Loss: 4.547722816467285
Epoch 1000, Loss: 4.204220294952393
Epoch 1100, Loss: 4.845988750457764
Epoch 1200, Loss: 3.663443088531494
Epoch 1300, Loss: 3.9273266792297363
Epoch 1400, Loss: 4.661615371704102
Epoch 1500, Loss: 3.8933355808258057
Epoch 1600, Loss: 4.836217403411865
Epoch 1700, Loss: 3.6647138595581055
Epoch 1800, Loss: 3.935274839401245
Epoch 1900, Loss: 3.707758665084839
Epoch 2000, Loss: 3.5942108631134033
Epoch 2100, Loss: 3.4407896995544434
Epoch 2200, Loss: 3.7329063415527344
Epoch 2300, Loss: 4.8374457359313965
Epoch 2400, Loss: 4.099700927734375
Epoch 2500, Loss: 4.904849052429199
Epoch 2600, Loss: 3.5542173385620117
Epoch 2700, Loss: 4.790956974029

In [ ]:
# 4. モデルによる文字生成
# モデルの評価モードに切り替え
model.eval()

# 初期の文字列を設定
start_str = "吾輩は"

# 文字列を生成する関数
def generate(model, start_str, max_len=100, temperature=1.0):
  # 文字列のリストをインデックスに変換
  input_str = [stoi.get(c, stoi[' ']) for c in start_str]
  # input_tensor = torch.tensor(input_str).unsqueeze(0)  # バッチサイズ１で入力
  input_tensor = torch.tensor(input_str).unsqueeze(0)

  generated_str = start_str  # 初期文字列を保存

  # 初期隠れ状態を設定
  h0 = None

  for _ in range(max_len):
    # モデルの出力を取得
    output, h0 = model(input_tensor, h0)

    # 出力の最後の時刻を取り出し、確率分布を計算
    logits = output[0, -1, :] / temperature
    probabilities = torch.softmax(logits, dim=-1)

    # 次の文字を確率的にサンプリング
    next_char_idx = torch.multinomial(probabilities, 1).item()

    # インデックスを文字に戻す
    # next_char = stoi[next_char_idx]
    next_char = itos.get(next_char_idx, ' ')

    # 生成された文字を追加
    generated_str += next_char

    # 次の入力として最後に生成した文字を使用
    input_tensor = torch.tensor([next_char_idx]).unsqueeze(0)

  return generated_str


In [ ]:
# 初期の文字列を設定
start_str = "吾輩は"

# 文字列を生成
generated_text = generate(model, start_str, max_len=100, temperature=0.7)

# 生成された文字列を表示
print("生成された文字列:")
print(generated_text)

生成された文字列:
吾輩はないのですから、机の前に置いても、今の顔があったと、豚だけた事に長ました。私はこの質は軽蔑なに借下女の胸を始めたと好いたら、まだ用のじゃないから、その時かになるいと云いた。これはそれな顔を見た時、おい


In [ ]:
import torch

# モデルの状態を保存する
torch.save(model.state_dict(), 'model_epoch10000.pt')


In [9]:
# トークナイザー
text = open("combined_clean.txt", encoding="utf-8").read()
chars = sorted(list(set(text)))
vocab_size = len(chars)

# 文字とIDの対応表
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

In [10]:
# エンコード:文字列を数値のリストに
def encode(s):
  return [stoi[c] for c in s]

# デコード:数値のリストを文字列に
def decode(l):
  return ''.join([itos[i] for i in l])

In [11]:
# 動作確認
sample = "私はだれ"
encoded = encode(sample)
decoded = decode(encoded)

print("元:", sample)
print("数値列:", encoded)
print("復元:", decoded)

元: 私はだれ
数値列: [1623, 68, 53, 97]
復元: 私はだれ


In [12]:
# テキスト全体を数値化
data = encode(text)  # 文字列 → ID列

In [15]:
# データを訓練用とテスト用に分ける
import numpy as np

data = np.array(data, dtype=np.int32)  # NumPy配列に変換(高速化のため)

n = int(0.9 * len(data))  # 90%を訓練用に
train_data = data[:n]
val_data = data[n:]

In [16]:
print("訓練データのサイズ:", len(train_data))
print("テストデータのサイズ:", len(val_data))
print("最初の20個:", train_data[:20])

訓練データのサイズ: 306173
テストデータのサイズ: 34020
最初の20個: [ 179   19  325 1505   61 1623    0  173    0   19 1623   68   50   67
  217  100  789   64  325 1505]


In [17]:
import torch

block_size = 16  # 入力の長さ（文脈サイズ）
batch_size = 4   # 同時に何個のデータを学習するか

def get_batch(split):
  data_split = train_data if split == 'train' else val_data
  ix = torch.randint(len(data_split) - block_size, (batch_size,))
  x = torch.stack([torch.tensor(data_split[i:i+block_size]) for i in ix])
  y = torch.stack([torch.tensor(data_split[i+1:i+1+block_size]) for i in ix])
  return x, y

In [18]:
xb, yb = get_batch('train')
print("入力(x):", xb.shape)
print("出力(y):", yb.shape)
print("xの中身:\n", xb)
print("yの中身:\n", yb)

入力(x): torch.Size([4, 16])
出力(y): torch.Size([4, 16])
xの中身:
 tensor([[1739,   28,   59,   28,   59,   20,   50,   97,  100,  771,   77,  983,
           97,   96,   61,   20],
        [  52,   21,  206,  217,   68,  625,   34,   63, 2254, 2430, 2262,   67,
         1241,   32,   94,   20],
        [  50,   40,   28,   94,   67, 1634,   64,   84,   47,   33,   32,   32,
           96,  223, 1032,   60],
        [  24,    0,   19,  325, 1505,   68,  173, 1124, 2350,  789,   67, 2063,
         1146,  704,   60,   27]], dtype=torch.int32)
yの中身:
 tensor([[  28,   59,   28,   59,   20,   50,   97,  100,  771,   77,  983,   97,
           96,   61,   20, 2463],
        [  21,  206,  217,   68,  625,   34,   63, 2254, 2430, 2262,   67, 1241,
           32,   94,   20,   23],
        [  40,   28,   94,   67, 1634,   64,   84,   47,   33,   32,   32,   96,
          223, 1032,   60,   27],
        [   0,   19,  325, 1505,   68,  173, 1124, 2350,  789,   67, 2063, 1146,
          704,   60,   27,